 ### Solving Capacitated Lot Sizing Problem using CPLEX's python module DOCPLEX

In [1018]:
import random as r
import numpy as np

r.seed(50)

no_of_items = 18
no_of_machines = 4
no_of_period = 6

# setting up the cost
low_setup_time = np.asarray([int(r.uniform(a=10,b=50)) for i in range(no_of_items*no_of_machines)])
high_setup_time = 1.5*low_setup_time

low_setup_cost = np.asarray([int(r.uniform(a=5,b=95)) for i in range(no_of_items*no_of_machines)])
high_setup_cost = 10*low_setup_cost

unit_production_cost = [round(r.uniform(a=1.5,b=2.5), 2) for i in range(no_of_items*no_of_machines)]
unit_inventory_cost = [round(r.uniform(a=0.2,b=0.4),2) for i in range(no_of_items*no_of_period)]
unit_production_time = [int(r.uniform(a=1,b=5)) for i in range(no_of_items*no_of_machines)]

demand = [round(r.uniform(a=1,b=180)) for i in range(no_of_items*no_of_period)]

In [1019]:
# Assigning Costs:

Item_list = [i for i in range(1,no_of_items+1)]
Machine_list = [j for j in range(1,no_of_machines+1)]
Period_list = [t for t in range(1,no_of_period+1)]

item_machine_pair = [(i,j) for i in Item_list for j in Machine_list]
item_period_pair = [(i,t) for i in Item_list for t in Period_list]
machine_period_pair = [(j,t) for j in Machine_list for t in Period_list]

Bij = {(i,j): s for (i,j),s in zip(item_machine_pair,unit_production_time)}   # Unit production time of item i on machine j;
Cij = {(i,j): s for (i,j),s in zip(item_machine_pair,unit_production_cost)}   # Unit production cost of item i on machine j;
Hit = {(i,t): s for (i,t),s in zip(item_period_pair,unit_inventory_cost)}     # Unit inventory cost of item i per period t;
Dit = {(i,t): s for (i,t),s in zip(item_period_pair,demand)}                  # Demand of item i at period t;
Fij = {(i,j): s for (i,j),s in zip(item_machine_pair,low_setup_time)}         # Setup time of item i on machine j;
Sij = {(i,j): s for (i,j),s in zip(item_machine_pair,low_setup_cost)}         # Setup cost of item i on machine j.

In [1020]:
# Instanciating the variables:

Vars = []
for items in Item_list:
    for machines in Machine_list:
        for periods in Period_list:
            Vars.append((items,machines,periods))
            
Inventory_vars = []
for items in Item_list:
    for periods in Period_list:
        Inventory_vars.append((items,periods))

In [1021]:
from docplex.mp.model import Model

In [1022]:
# Initializing the Model:

mip = Model('Capacitated Lot Sizing Problem')

In [1023]:
# Declaring decision variables:

X = mip.integer_var_dict(Vars, lb=0, name='x')
Y = mip.binary_var_dict(Vars,name='y')
S = mip.integer_var_dict(Inventory_vars, lb=0, name='s')

In [1024]:
# Averaging out demand of each item per period in order to calculate capacity of each machine per period:

Cap = mip.sum(((Dit[i,t]/no_of_machines)*Bij[i,j]) + Fij[i,j] for i,j,t in X)                       
capacity_per_period = []
for i in range(1,no_of_period+1):
    Q = int((Cap/(no_of_machines*i)).constant)      
    capacity_per_period.append(Q)

Qjt = {(j,t): s for (j,t),s in zip(machine_period_pair,capacity_per_period*no_of_period)}           # Capacity of machine j at period t;
M = max(capacity_per_period)                                                                        # Maximum capacity of machines per period.

In [1025]:
# Constraints:

# constraint_1 ensure that the demand of each item in each period should be satisfied 
constraint_1 = mip.add_constraints(mip.sum(X[i,j,t] for j in range(1,no_of_machines+1)) + (-S[i,t] + S[i,t-1] if t-1 > 0 else -S[i,t]) == Dit[i,t] for i,t in S)

# constraint_2 restrict the production capacity for each machine j at each period t
constraint_2 = mip.add_constraints(mip.sum(Bij[i,j]*X[i,j,t] + Fij[i,j]*Y[i,j,t] for i in range(1,no_of_items+1)) <= Qjt[j,t] for j,t in Qjt)

# constraint_3 impose the setup cost on machine j at period t for item i when Xijt is positive.
constraint_3 = mip.add_constraints(X[i,j,t] <= M*Y[i,j,t] for i,j,t in X)

In [1026]:
# Objective function:

setup_cost = mip.sum(Sij[i,j]*Y[i,j,t] for i,j,t in Y) 
production_cost = mip.sum(Cij[i,j]*X[i,j,t] for i,j,t in X)
inventory_cost = mip.sum(Hit[i,t]*S[i,t] for i,t in S)
total_cost = setup_cost + production_cost + inventory_cost

In [1027]:
# Goal:

mip.minimize(total_cost)

In [1028]:
mip.print_information()

Model: Capacitated Lot Sizing Problem
 - number of variables: 972
   - binary=432, integer=540, continuous=0
 - number of constraints: 564
   - linear=564
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [1029]:
solution = mip.solve(log_output=True)
assert solution is not None
value = solution.objective_value

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 18 rows and 0 columns.
MIP Presolve added 36 rows and 0 columns.
MIP Presolve modified 900 coefficients.
Reduced MIP has 582 rows, 972 columns, and 2394 nonzeros.
Reduced MIP has 432 binaries, 540 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.96 ticks)
Found incumbent of value 57494.400000 after 0.00 sec. (6.44 ticks)
Probing fixed 0 vars, tightened 36 bounds.
Probing time = 0.01 sec. (1.18 ticks)
Cover probing fixed 0 vars, tightened 15 bounds.
Tried aggregator 1 time.
MIP Presolve modified 54 coefficients.
Reduced MIP has 582 rows, 972 columns, and 2394 nonzeros.
Reduced MIP has 432 binaries, 540 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.79 ticks)
Probing fixed 0 vars, tightened 3 bounds.
Probing time = 0.02 sec. (1.11 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using

In [1030]:
print("The objective function value is: {}".format(round(value,2)))

The objective function value is: 19370.49


In [1031]:
print("The Solution details are:\n{}and it was solved by: {} solver".format(solution.solve_details,solution.solved_by))

The Solution details are:
status  = integer optimal, tolerance
time    = 0.391 s.
problem = MILP
gap     = 0.00948505%
and it was solved by: cplex_local solver


In [1032]:
solution_y = solution.get_value_dict(Y)
nonzero_allocation = []
for keys in solution_y:
    if solution_y[keys] == 1:
        nonzero_allocation.append(keys)
    else:
        pass
solution_x = solution.get_value_dict(X)
count = []
for keys in solution_x:
    if solution_x[keys] != 0:
        count.append(keys)
    else:
        pass
if nonzero_allocation == count: print("All cases matched. Hurray!")

All cases matched. Hurray!


In [1033]:
# solution.export('Output')